In [1]:
from datetime import datetime
import logging

import pandas as pd
import numpy as np
import sqlite3

# general idea was to try to figure out how to avoid storing duplicate records to postgres even if they have a different timestamp
# it's a huge pain in the ass, the way im doing it works just fine rn so fk this.

In [2]:
connection_obj = sqlite3.connect('prac.db',
                             detect_types=sqlite3.PARSE_DECLTYPES |
                             sqlite3.PARSE_COLNAMES)

In [3]:
df2 = pd.DataFrame(np.array([['email1@gmail.com', 'email2@gmail.com', 'email3@gmail.com'],
                             ['name1', 'name2', 'name2'],
                             ['lastname1', 'lastname2', 'lastname3']]
                            ),
                   columns=['email', 'first_name', 'last_name'])
df2['scrape_date'] = datetime.now()

In [4]:
cursor_obj = connection_obj.cursor()
cursor_obj.execute("DROP TABLE IF EXISTS practice_table")

table = """ CREATE TABLE practice_table (
            email VARCHAR(255) NOT NULL,
            first_name CHAR(25) NOT NULL,
            last_name CHAR(25) NOT NULL,
            scrape_date TIMESTAMP NOT NULL
        ); """
 
cursor_obj.execute(table)

In [20]:
df2.to_sql('practice_table', connection_obj, index = False, if_exists = 'append')
df2.to_sql('practice_table', connection_obj, index = False, if_exists = 'append')
df2.to_sql('practice_table', connection_obj, index = False, if_exists = 'append')

In [46]:
query = 'SELECT * FROM practice_table;'
query_df = pd.read_sql(query, connection_obj)

In [45]:
data = ["email1@gmail.com", "name1", "lastname1", datetime.now()]
cursor_obj.execute("""INSERT OR REPLACE INTO practice_table
    VALUES (?, ?, ?, ?);""",
    data)
connection_obj.commit()

In [42]:
data = ["email1@gmail.com", "name1", "lastname1", datetime.now()]
cursor_obj.execute("""INSERT INTO practice_table (email, first_name, last_name, scrape_date)
    SELECT email, first_name, last_name, scrape_date
    WHERE NOT EXISTS (SELECT 1 FROM practice_table WHERE email = email, first_name = first_name, last_name = last_name;""",
    data)
connection_obj.commit()

OperationalError: near ",": syntax error

In [24]:
query = 'SELECT * FROM practice_table;'
query_df = pd.read_sql(query, connection_obj)

In [20]:
rows = list(df2.values.tolist())
cursor_obj.executemany('insert into practice_table values (?,?,?,?)', rows)

In [75]:
list(df2.values.tolist())

[['email1@gmail.com',
  'email2@gmail.com',
  'email3@gmail.com',
  Timestamp('2022-04-30 11:20:21.448396')],
 ['name1', 'name2', 'name2', Timestamp('2022-04-30 11:20:21.448396')],
 ['lastname1',
  'lastname2',
  'lastname3',
  Timestamp('2022-04-30 11:20:21.448396')]]

In [9]:
cursor_obj.close()